<a href="https://colab.research.google.com/github/Jesuslovesyou21/bigdata/blob/main/prac2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ 필요한 라이브러리 임포트
from datasets import load_dataset, concatenate_datasets
from transformers import BertTokenizerFast, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
import torch

# ✅ 1. IMDB 영화 리뷰 데이터셋 로드
# IMDB는 'train'과 'test'로 나뉜 binary classification용 감성 데이터셋
dataset = load_dataset("imdb")

# ✅ 2. 과적합 방지를 위한 균형잡힌 샘플링
# 아래 방식으로 데이터 훈련/테스트 세트 선택 할 경우 데이터 편향으로 인한 과적합 또는 오적합 발생
# train_data = dataset["train"].select(range(2000))  # 메모리 초과 방지용
# test_data = dataset["test"].select(range(500))     # 메모리 초과 방지용
# 원본 train 데이터는 부정(0) 리뷰가 앞쪽에 몰려 있음 ⇒ 모델 편향 유발
# 따라서 부정/긍정 각각 5000개씩 무작위 추출하여 균형 유지
neg_samples = dataset["train"].filter(lambda x: x['label'] == 0).shuffle(seed=42).select(range(5000))
pos_samples = dataset["train"].filter(lambda x: x['label'] == 1).shuffle(seed=42).select(range(5000))

# 부정/긍정 결합 후 다시 셔플하여 train 데이터 구성
balanced_dataset = concatenate_datasets([neg_samples, pos_samples]).shuffle(seed=42)

# ✅ 3. 학습/평가용 데이터 분리
# 학습 데이터: 균형 샘플 중 2,000개 사용 (Colab 환경 고려)
# 테스트 데이터: 원본 IMDB test 데이터 중 500개 사용
train_data = balanced_dataset.select(range(2000))
test_data = load_dataset("imdb")["test"].select(range(500))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
# ✂️ 훈련 데이터셋에서 리뷰 텍스트와 라벨만 분리하여 별도 변수에 저장
train_texts = train_data["text"]   # 리뷰 본문 텍스트 목록 (리스트 형태)
train_labels = train_data["label"] # 감성 레이블 (0: 부정, 1: 긍정)

# 🔤 사전학습된 BERT용 토크나이저 로드 (WordPiece 기반 토큰화)
# 'bert-base-uncased'는 모든 텍스트를 소문자로 처리함
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# ✂️ 최대 시퀀스 길이 설정 (너무 긴 문장은 자르고, 짧은 문장은 패딩함)
MAX_LEN = 256

# 🧪 토크나이저로 문장들을 BERT 입력 형태로 변환
# truncation=True → 길면 자르기 / padding='max_length' → 짧으면 256길이에 맞춰 패딩
# return_tensors="pt" → PyTorch 텐서 형태로 반환
encodings = tokenizer(
    train_texts,
    truncation=True,
    padding="max_length",
    max_length=MAX_LEN,
    return_tensors="pt"
)

# 🔠 인코딩 결과에서 input_ids와 attention_mask 추출
# input_ids: 토큰의 정수 인덱스
# attention_mask: 패딩이 아닌 부분은 1, 패딩된 부분은 0
train_inputs = encodings["input_ids"]
train_masks = encodings["attention_mask"]

# 🏷️ 라벨도 torch 텐서 형태로 변환 (모델 학습에 사용하기 위함)
train_labels = torch.tensor(train_labels)

In [ ]:
# 🧳 PyTorch Dataset 및 Dataloader 구성

# 한 번에 학습할 데이터 수 설정 (미니 배치 크기)
batch_size = 16

# 📦 TensorDataset: 입력 데이터(input_ids), 마스크(attention_mask), 정답 라벨(labels)을 하나의 Dataset으로 묶음
# => 학습 시 각 인덱스에 대해 (input_ids, attention_mask, label)을 반환
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)

# 🚚 DataLoader: 실제 학습에서 배치 단위로 데이터를 공급해주는 역할
# RandomSampler: 매 epoch마다 데이터를 **무작위로 섞어** 샘플링 → 학습 성능 향상에 도움
# batch_size: 한 배치에 포함될 샘플 수
train_dataloader = DataLoader(
    train_dataset,                     # 학습할 데이터셋
    sampler=RandomSampler(train_dataset),  # 랜덤 샘플링 (epoch마다 데이터 순서 섞기)
    batch_size=batch_size             # 한 번에 읽어올 샘플 개수 (배치 단위)
)

In [ ]:
# 🔧 GPU 사용 여부 확인
# CUDA가 사용 가능하면 GPU 사용, 그렇지 않으면 CPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🧠 사전학습된 BERT 분류 모델 로딩
# 'bert-base-uncased': 소문자화된 BERT 사전학습 모델
# num_labels=2: 이진 분류 문제 (긍정/부정)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# 모델을 선택한 장치(GPU 또는 CPU)로 이동
model.to(device)

# 🔁 옵티마이저 및 학습률 스케줄러 설정
# AdamW: BERT 논문에서 권장하는 옵티마이저 (가중치 감쇠 적용)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# 총 학습 epoch 수 설정
epochs = 2

# 전체 학습 스텝 수 계산 (총 배치 수 × epoch 수)
total_steps = len(train_dataloader) * epochs

# 학습률 스케줄러 설정
# Warm-up 없이 선형 감소 방식으로 학습률 조정
scheduler = get_linear_schedule_with_warmup(
    optimizer,                 # 사용할 옵티마이저
    num_warmup_steps=0,        # warm-up 단계 없음
    num_training_steps=total_steps  # 전체 학습 스텝 수
)

In [ ]:
import time
import datetime
import numpy as np
import random

# 🧬 랜덤 시드 고정 (결과 재현 가능성을 높이기 위함)
seed_val = 777
random.seed(seed_val)                    # 파이썬 random 고정
np.random.seed(seed_val)                # 넘파이 random 고정
torch.manual_seed(seed_val)             # PyTorch random 고정
torch.cuda.manual_seed_all(seed_val)    # CUDA에서도 동일하게 시드 고정

# 학습 시간 포맷 함수 (초 → 시:분:초)
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round(elapsed))))

# 🏋️ 모델 학습 시작
model.train()  # 모델을 학습 모드로 전환 (Dropout, BatchNorm 등이 학습 모드로 작동)

# 각 epoch 반복
for epoch_i in range(epochs):
    print(f"\n======== Epoch {epoch_i + 1} / {epochs} ========")
    t0 = time.time()  # 학습 시작 시간 저장
    total_loss = 0    # 누적 손실 초기화

    # 미니배치 반복
    for step, batch in enumerate(train_dataloader):
        # 매 100스텝마다 경과 시간 출력
        if step % 100 == 0:
            elapsed = format_time(time.time() - t0)
            print(f"  ▶ Step {step}/{len(train_dataloader)} - Elapsed: {elapsed}")

        # 입력 데이터와 라벨을 장치(GPU/CPU)로 이동
        b_input_ids, b_input_mask, b_labels = [b.to(device) for b in batch]

        model.zero_grad()  # 기울기 초기화

        # 🔁 순전파 → 손실 계산
        outputs = model(
            b_input_ids,
            attention_mask=b_input_mask,
            labels=b_labels
        )
        loss = outputs.loss         # CrossEntropyLoss 포함
        total_loss += loss.item()   # 손실 누적

        # 🔄 역전파 → 가중치 갱신
        loss.backward()  # 손실 기준으로 역전파 수행

        # 그래디언트 클리핑: exploding gradient 방지
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()  # 가중치 업데이트
        scheduler.step()  # 학습률 조정

    # ⏱ 에폭 단위 평균 손실 출력
    avg_loss = total_loss / len(train_dataloader)
    print(f"✅ Epoch {epoch_i+1} Avg Loss: {avg_loss:.4f}")
    print(f"🕒 Epoch time: {format_time(time.time() - t0)}")

In [ ]:
from transformers import pipeline

# 📍 GPU 사용 가능 여부 확인 → GPU가 있으면 0, 없으면 CPU(-1)로 설정
device_id = 0 if torch.cuda.is_available() else -1

# 🤖 학습된 모델로 파이프라인 구성
clf_pipeline = pipeline(
    task="text-classification",        # 작업 유형: 문장 분류
    model=model,                       # fine-tuning된 BERT 분류 모델
    tokenizer=tokenizer,               # 같은 base의 토크나이저 사용
    device=device_id,                  # 디바이스 설정: GPU or CPU
    return_all_scores=True,            # 두 클래스의 확률 모두 출력
    function_to_apply="softmax"        # 출력 로짓 → 확률값으로 변환
)

# 🔍 예측할 테스트 문장 리스트
sample_texts = [
    "This movie was absolutely fantastic. I loved the plot and the characters.",   # 긍정
    "It was the worst film I have ever seen. Completely boring.",                  # 부정
    "Not bad, but not great either. Just okay."                                    # 중립에 가까움 (판단 애매)
]

# 🧾 파이프라인을 통해 예측 수행
predictions = clf_pipeline(sample_texts)

# 📋 각 문장별 예측 결과 출력
for text, result in zip(sample_texts, predictions):
    print(f"\n📌 입력 문장: {text}")
    for label in result:
        print(f"  {label['label']} → {label['score']:.4f}")  # label: POSITIVE/NEGATIVE, score: 확률값

In [ ]:
Device set to use cuda:0

📌 입력 문장: This movie was absolutely fantastic. I loved the plot and the characters.
  LABEL_0 → 0.0199
  LABEL_1 → 0.9801

📌 입력 문장: It was the worst film I have ever seen. Completely boring.
  LABEL_0 → 0.9772
  LABEL_1 → 0.0228

📌 입력 문장: Not bad, but not great either. Just okay.
  LABEL_0 → 0.3401
  LABEL_1 → 0.6599

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# 🎯 테스트셋 전처리
test_texts = test_data["text"]             # 테스트 문장 리스트
test_labels = test_data["label"]           # 테스트 레이블 리스트

# 🧪 토크나이징 및 인코딩 (훈련과 동일하게 max_length 기준 padding/truncation 수행)
test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding="max_length",
    max_length=MAX_LEN,
    return_tensors="pt"                    # PyTorch 텐서 형태로 반환
)

test_inputs = test_encodings["input_ids"]         # 입력 토큰 ID
test_masks = test_encodings["attention_mask"]     # 어텐션 마스크
test_labels_tensor = torch.tensor(test_labels)    # 정답 레이블

# 📦 PyTorch용 테스트 Dataset 및 DataLoader 구성
test_dataset = TensorDataset(test_inputs, test_masks, test_labels_tensor)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

# 📈 평가 함수 정의
def evaluate_model(model, dataloader):
    model.eval()              # 평가 모드 (Dropout/BatchNorm 등 비활성화)
    preds = []                # 예측값 저장용 리스트
    true_labels = []          # 실제값 저장용 리스트

    for batch in dataloader:
        # 미니배치 데이터를 GPU/CPU로 이동
        b_input_ids, b_input_mask, b_labels = [b.to(device) for b in batch]

        with torch.no_grad():  # 그래디언트 계산 비활성화 (속도↑, 메모리↓)
            outputs = model(b_input_ids, attention_mask=b_input_mask)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).cpu().numpy()  # 가장 확률 높은 클래스 선택
        labels = b_labels.cpu().numpy()

        preds.extend(predictions)      # 예측 결과 누적
        true_labels.extend(labels)     # 실제 레이블 누적

    return preds, true_labels

# 🧪 평가 실행
preds, true_labels = evaluate_model(model, test_dataloader)

# ✅ 정확도 및 F1 점수 계산 및 출력
acc = accuracy_score(true_labels, preds)                         # 정확도: 예측과 실제 일치 비율
f1 = f1_score(true_labels, preds, average='weighted')            # F1 점수: 정밀도와 재현율의 조화 평균

print(f"\n✅ Test Accuracy: {acc:.4f}")
print(f"✅ Test F1 Score: {f1:.4f}")